In [1]:
!pip install --quiet diffusers transformers accelerate ftfy streamlit pillow pyngrok

In [2]:
!pip install pyngrok

In [3]:
!pip install gradio

In [4]:
from huggingface_hub import login
login("your_key")


In [5]:
import torch
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler
from transformers import BlipProcessor, BlipForConditionalGeneration
import gradio as gr
import openai

# Load models
def load_models():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    dtype = torch.float16 if device == "cuda" else torch.float32

    # Load Stable Diffusion
    sd_pipe = StableDiffusionPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        torch_dtype=dtype
    )
    sd_pipe.scheduler = EulerDiscreteScheduler.from_config(sd_pipe.scheduler.config)
    sd_pipe = sd_pipe.to(device)

    # Load BLIP
    blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    blip_model = BlipForConditionalGeneration.from_pretrained(
        "Salesforce/blip-image-captioning-base"
    ).to(device).eval()

    return sd_pipe, blip_processor, blip_model, device

# Enrich prompt manually
def enrich_prompt(prompt):
    return f"A highly detailed, photorealistic image of {prompt}, 4k, vibrant lighting, cinematic atmosphere"

# Generate image with negative prompt
def generate_image(prompt, sd_pipe):
    enriched_prompt = enrich_prompt(prompt)
    negative_prompt = "blurry, low resolution, distorted, bad anatomy, extra limbs, poorly drawn face, disfigured, mutated"
    image = sd_pipe(
        enriched_prompt,
        num_inference_steps=50,
        guidance_scale=9.0,
        negative_prompt=negative_prompt
    ).images[0]
    return image

# Generate caption from image
def generate_caption(image, blip_processor, blip_model, device):
    inputs = blip_processor(image, return_tensors="pt").to(device)
    with torch.no_grad():
        out = blip_model.generate(**inputs)
    caption = blip_processor.decode(out[0], skip_special_tokens=True)
    return caption

# Expand caption using GPT
def enrich_caption_with_gpt(base_caption):
    prompt = (
        "Expand this image description into a detailed, vivid, and poetic caption that captures "
        "all visual elements, atmosphere, colors, and mood:\n\n"
        f"'{base_caption}'"
    )
    try:
        response = openai.Completion.create(
            engine="text-davinci-003",
            prompt=prompt,
            max_tokens=100,
            temperature=0.85,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
        )
        detailed_caption = response.choices[0].text.strip()
        return detailed_caption
    except Exception as e:
        print(f"OpenAI API error: {e}")
        return base_caption

def generate_detailed_caption(image, blip_processor, blip_model, device):
    base_caption = generate_caption(image, blip_processor, blip_model, device)
    detailed_caption = enrich_caption_with_gpt(base_caption)
    return detailed_caption

# Full pipeline: prompt → image + caption
def full_pipeline(prompt):
    image = generate_image(prompt, sd_pipe)
    caption = generate_caption(image, blip_processor, blip_model, device)
    return image, caption

# Wrapper for uploaded image captioning
def caption_wrapper(image):
    return generate_caption(image, blip_processor, blip_model, device)

# Initialize models
sd_pipe, blip_processor, blip_model, device = load_models()

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## 🖼 Enhanced Image Generator & Captioning App")
    gr.Markdown("Generate accurate, high-quality images or captions with AI.")

    with gr.Tabs():
        # Tab 1 - Text to Image
        with gr.TabItem("1. Generate Image from Text"):
            with gr.Row():
                prompt_input = gr.Textbox(label="Enter Prompt", placeholder="e.g. A cat astronaut exploring Mars")
                generate_btn = gr.Button("Generate Image")
            image_output = gr.Image(label="Generated Image", interactive=True)
            caption_output = gr.Textbox(label="Generated Caption")
            generate_btn.click(fn=full_pipeline, inputs=prompt_input, outputs=[image_output, caption_output])

        # Tab 2 - Image Captioning
        with gr.TabItem("2. Caption Uploaded Image"):
            image_input = gr.Image(type="pil", label="Upload Image")
            caption_btn = gr.Button("Generate Caption")
            caption_result = gr.Textbox(label="Caption")
            caption_btn.click(fn=caption_wrapper, inputs=image_input, outputs=caption_result)

# Launch app
demo.launch(share=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://16d6d76fe9dcb767df.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
